# Tree

부모-자녀 관계로 데이터 관리를 용이하게 한 구조

`ex.`폴더를 기반으로 한 파일 시스템이 대표적, XML

### XML

- XML 은 한번 가공(XML Beautifier)해줘야 Tree 형식이 된다
- pandas 에서 모든 <age> 를 한번에 불러올 수 있다(태그별 호출)
- 태그 이름은 정하기 나름이다(줄여서 쓰는건 옛날 방식)

#### XML - Json
- XML 이 Json 보다 더 많은 정보를 담을 수 있다
- json 파일로 변환할 수 있다

# LocationIQ 로 위도, 경도 구하기
LocationIQ - Free & Fast Geocoding and Reverse Geocoding service

In [2]:
import os

In [6]:
LOCATIONIQ_TOKEN = os.environ['locationiq_token']
LOCATIONIQ_TOKEN

'ba4fed066fb890'

In [7]:
import requests
requests.__version__

'2.21.0'

# API 호출
parameter 이용

cf. 카카오맵 API : header
    보안수준이 더 높다

In [8]:
# LocationIQ로 “서울창업허브” 검색

# 원래 URL: https://us1.locationiq.com/v1/search.php?key=블라블라&q=서울창업허브&format=json
# URL을 물음표(?)를 중심으로 두 부분으로 나눕니다.

# 1. 물음표 앞부분.
url = 'https://us1.locationiq.com/v1/search.php'

# 2. 물음표 뒷부분.
payload = {
    'key': LOCATIONIQ_TOKEN,
    'q': '서울창업허브',
    'format': 'json'
}

# API 호출
response = requests.get(url, params=payload)

print(response)

<Response [200]>


Response(응답)는 여러 내용을 담고 있습니다.

가장 기본적인 게 바로 HTTP Status Code 입니다.

<https://en.wikipedia.org/wiki/List_of_HTTP_status_codes>

Response code|의미
-|-
200 | 성공
201 | 성공
404 | 찾을 수 없다
500 | 뭔가 잘못 됐다

- 200번대는 다 성공

In [9]:
# Response의 HTTP Status Code 확인

print(response.status_code)

200


Json 도 XML 과 마찬가지로 Tree 가 되기 위한 재료

In [11]:
# Response의 Text 확인

print(response.text)

[{"place_id":"206185511","licence":"https:\/\/locationiq.com\/attribution","osm_type":"way","osm_id":"608088218","boundingbox":["37.5463183","37.5468967","126.9492191","126.9501552"],"lat":"37.54662875","lon":"126.949684293519","display_name":"Seoul Startup Hub, Baekbeom-ro 31-gil, Ahyeon-dong, Mapo-gu, Seoul, 04140, South Korea","class":"office","type":"government","importance":0.001}]


좀더 보기 좋게 JSON Beautifier로 바꿔봅니다.

<https://www.jsonformatter.io/>

In [ ]:
[
   {
      "place_id": "206185511",
      "licence": "https://locationiq.com/attribution",
      "osm_type": "way",
      "osm_id": "608088218",
      "boundingbox": [
         "37.5463183",
         "37.5468967",
         "126.9492191",
         "126.9501552"
      ],
      "lat": "37.54662875",
      "lon": "126.949684293519",
      "display_name": "Seoul Startup Hub, Baekbeom-ro 31-gil, Ahyeon-dong, Mapo-gu, Seoul, 04140, South Korea",
      "class": "office",
      "type": "government",
      "importance": 0.001
   }
]

In [14]:
results = response.json()
results

[{'place_id': '206185511',
  'licence': 'https://locationiq.com/attribution',
  'osm_type': 'way',
  'osm_id': '608088218',
  'boundingbox': ['37.5463183', '37.5468967', '126.9492191', '126.9501552'],
  'lat': '37.54662875',
  'lon': '126.949684293519',
  'display_name': 'Seoul Startup Hub, Baekbeom-ro 31-gil, Ahyeon-dong, Mapo-gu, Seoul, 04140, South Korea',
  'class': 'office',
  'type': 'government',
  'importance': 0.001}]

In [15]:
# 리스트 원소의 갯수
len(results)

1

In [16]:
# 리스트의 첫번째 값
results[0]

{'place_id': '206185511',
 'licence': 'https://locationiq.com/attribution',
 'osm_type': 'way',
 'osm_id': '608088218',
 'boundingbox': ['37.5463183', '37.5468967', '126.9492191', '126.9501552'],
 'lat': '37.54662875',
 'lon': '126.949684293519',
 'display_name': 'Seoul Startup Hub, Baekbeom-ro 31-gil, Ahyeon-dong, Mapo-gu, Seoul, 04140, South Korea',
 'class': 'office',
 'type': 'government',
 'importance': 0.001}

In [17]:
# 위도
results[0]['lat']

'37.54662875'

In [19]:
# 경도
results[0]['lon']

'126.949684293519'

# DataFrame 만들기
XML 보다 Json 이 훨씬 빠르다

In [20]:
import pandas as pd

In [22]:
columns = results[0].keys()
columns

dict_keys(['place_id', 'licence', 'osm_type', 'osm_id', 'boundingbox', 'lat', 'lon', 'display_name', 'class', 'type', 'importance'])

In [23]:
pd.DataFrame(results, columns=columns)

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance
0,206185511,https://locationiq.com/attribution,way,608088218,"[37.5463183, 37.5468967, 126.9492191, 126.9501...",37.54662875,126.949684293519,"Seoul Startup Hub, Baekbeom-ro 31-gil, Ahyeon-...",office,government,0.001


완전 틀린 주소를 찾으려고 하면 404 Not Found 에러를 얻게 됩니다.

In [24]:
# 완전 틀린 주소를 찾을 때

url = 'https://us1.locationiq.com/v1/search.php'
payload = {
    'key': LOCATIONIQ_TOKEN,
    'q': '이 세계 어느 곳에도 없는 주소',
    'format': 'json'
}

response = requests.get(url, params=payload)

print(response)

<Response [404]>


In [25]:
print(response.text)

{"error":"Unable to geocode"}


# 에러인지 확인하는 방법
1. 코드에 넣어서 실행해보면 정보가 안 나온다
2. Response code 가 200 이 아니다
3. response.text 를 실행해봤는데 error 가 나온다